
## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension
%matplotlib inline


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
!pip install gmaps

In [3]:
#!conda install -c conda-forge gmaps
#!conda install -c conda-forge/label/gcc7 gmaps 


In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,58.48,71,47,4.76,scattered clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-5.93,94,100,3.51,overcast clouds
2,2,Ha Tinh,VN,18.3333,105.9000,85.98,65,43,8.59,scattered clouds
3,3,Castro,BR,-24.7911,-50.0119,60.12,97,28,1.68,scattered clouds
4,4,Ozernovskiy,RU,51.5000,156.5167,30.18,66,100,23.42,overcast clouds


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=int(input('Enter the minimum vacation temperature desired [deg F]'))
max_temp=int(input('Enter the maximum vacation temperature desired [deg F]'))

Enter the minimum vacation temperature desired [deg F]5
Enter the maximum vacation temperature desired [deg F]30


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df=city_data_df.loc[(city_data_df['Max Temp']<max_temp) & (city_data_df['Max Temp']>min_temp)]
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Thompson,CA,55.7435,-97.8558,10.56,61,100,17.27,overcast clouds
11,11,Dikson,RU,73.5069,80.5464,23.99,91,99,7.61,overcast clouds
18,18,Chokurdakh,RU,70.6333,147.9167,5.40,87,6,5.93,clear sky
23,23,Khatanga,RU,71.9667,102.5000,9.00,90,100,10.25,overcast clouds
49,49,Deputatskiy,RU,69.3000,139.9000,6.60,90,100,4.29,overcast clouds


In [42]:
# 4a. Determine if there are any empty rows.
#city_data_df
city_data_df.isnull().sum()

WeatherPy_Database.csv    0
City                      0
Country                   0
Lat                       0
Lng                       0
Max Temp                  0
Humidity                  0
Cloudiness                0
Wind Speed                0
Current Description       0
dtype: int64

In [43]:

# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## A new DataFrame is created based on the minimum and maximum temperature, and empty rows are dropped. (5 pt) ##
try:
    clean_df=city_data_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_df=city_data_df

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Thompson,CA,10.56,overcast clouds,55.7435,-97.8558,
11,Dikson,RU,23.99,overcast clouds,73.5069,80.5464,
18,Chokurdakh,RU,5.40,clear sky,70.6333,147.9167,
23,Khatanga,RU,9.00,overcast clouds,71.9667,102.5000,
49,Deputatskiy,RU,6.60,overcast clouds,69.3000,139.9000,
50,Flin Flon,CA,18.82,overcast clouds,54.7682,-101.8650,
59,Sangar,RU,28.96,light snow,63.9242,127.4739,
63,Ostrovnoy,RU,19.76,overcast clouds,68.0531,39.5131,
67,Longyearbyen,SJ,17.44,broken clouds,78.2186,15.6401,
74,Great Falls,US,16.25,overcast clouds,47.5002,-111.3008,


In [47]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    Lat=row['Lat']
    Lng=row['Lng']
    
    params["location"] = f"{Lat},{Lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    results = hotels['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index,"Hotel Name"]=results[0]['name']
    except (IndexError):
        print("Missing field/result... skipping.")
       

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [48]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"].astype(bool)]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Thompson,CA,10.56,overcast clouds,55.7435,-97.8558,Burntwood Hotel
23,Khatanga,RU,9.00,overcast clouds,71.9667,102.5000,"""Khatanga"""
49,Deputatskiy,RU,6.60,overcast clouds,69.3000,139.9000,Baza Otdykha
50,Flin Flon,CA,18.82,overcast clouds,54.7682,-101.8650,Victoria Inn Flin Flon
67,Longyearbyen,SJ,17.44,broken clouds,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
74,Great Falls,US,16.25,overcast clouds,47.5002,-111.3008,La Quinta Inn & Suites by Wyndham Great Falls
89,Pevek,RU,8.73,overcast clouds,69.7008,170.3133,Severnoye Zoloto
90,Talnakh,RU,29.21,light snow,69.4865,88.3972,Talnakh Hotel
123,High Prairie,CA,17.87,overcast clouds,55.4334,-116.4858,Royal Star Motor Inn
128,Cheyenne,US,22.77,clear sky,41.1400,-104.8203,The Historic Plains Hotel & Restaurant


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Weather</b></dt><dd>{Current Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
locations =clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))